### Data analysis

In [1]:
# Reload data
import pandas as pd

resultsDirectory = "../results/results-20250308/"
resultsFilename = "dfresults.csv-clean.csv"

df_results = pd.read_csv(resultsDirectory+resultsFilename,index_col=0)

In [2]:
df_results.columns

Index(['Question Set', 'Modality Type', 'Question de', 'Question fr',
       'Question it', 'Answer A de', 'Answer A fr', 'Answer A it',
       'Answer B de', 'Answer B fr', 'Answer B it', 'Answer C de',
       'Answer C fr', 'Answer C it', 'Answer D de', 'Answer D fr',
       'Answer D it', 'Answer E de', 'Answer E fr', 'Answer E it',
       'MC modality', 'EngPrompt', 'Model', 'Run', 'DE', 'FR', 'IT',
       'DE_clean', 'FR_clean', 'IT_clean', 'DE_correct', 'FR_correct',
       'IT_correct', 'AnswerType',
       'Category (1=General medicine, 2=Surgery/Traumatology, 3=Pediatrics, 4=gynocology, 5=Public Health and others',
       'acute (=1) vs chronic (=0)',
       'Diagnosis (=1), Treatment (=2), Other(=3)'],
      dtype='object')

### Interpret data statistically with a logistic mixed effects model



In [3]:
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.cov_struct import Exchangeable, Independence
from statsmodels.genmod.generalized_linear_model import GLM



In [4]:
def reshape_to_long(df):
    lang_cols = ['DE_correct', 'FR_correct', 'IT_correct']
    existing_lang_cols = [c for c in lang_cols if c in df.columns]
    id_cols = [c for c in df.columns if c not in existing_lang_cols]
    
    df_long = pd.melt(
        df,
        id_vars=id_cols,
        value_vars=existing_lang_cols,
        var_name='Language',
        value_name='Correct'
    )
    
    df_long['Language'] = df_long['Language'].str.replace('_correct', '')
    
    df_long['Item'] = df_long['Question Set'].astype(str)
    
    return df_long

In [5]:
# Remove irrelevant columns
df_results_for_analysis = df_results.loc[:,('Question Set',
       'EngPrompt', 'Model', 'Run', 'DE_correct', 'FR_correct',
       'IT_correct', 'AnswerType')] 
df_results_for_analysis['Category'] = df_results['Category (1=General medicine, 2=Surgery/Traumatology, 3=Pediatrics, 4=gynocology, 5=Public Health and others']
df_results_for_analysis['Acute'] = df_results['acute (=1) vs chronic (=0)']
df_results_for_analysis['Diagnosis'] = df_results['Diagnosis (=1), Treatment (=2), Other(=3)']

df_results_for_analysis

,Question Set,EngPrompt,Model,Run,DE_correct,FR_correct,IT_correct,AnswerType,Category,Acute,Diagnosis
Unique ID,,,,,,,,,,,
2,Set1.2,True,llama33-70b,0,True,True,True,0,1,0,1
3,Set1.3,True,llama33-70b,0,True,True,True,0,2,1,1
7,Set1.7,True,llama33-70b,0,True,True,True,0,1,0,2
8,Set1.8,True,llama33-70b,0,True,True,True,0,1,1,1
9,Set1.9,True,llama33-70b,0,False,True,False,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...
145,Set3.45,False,gpt4o,9,True,True,True,0,1,1,1
146,Set3.46,False,gpt4o,9,True,True,True,0,1,1,2
148,Set3.48,False,gpt4o,9,True,True,True,0,2,1,2


In [6]:
df_results_for_analysis.to_csv("all_quantitative_results.csv")

In [6]:
df_long = reshape_to_long(df_results_for_analysis)
df_long

,Question Set,EngPrompt,Model,Run,AnswerType,Category,Acute,Diagnosis,Language,Correct,Item
0,Set1.2,True,llama33-70b,0,0,1,0,1,DE,True,Set1.2
1,Set1.3,True,llama33-70b,0,0,2,1,1,DE,True,Set1.3
2,Set1.7,True,llama33-70b,0,0,1,0,2,DE,True,Set1.7
3,Set1.8,True,llama33-70b,0,0,1,1,1,DE,True,Set1.8
4,Set1.9,True,llama33-70b,0,0,3,0,1,DE,False,Set1.9
...,...,...,...,...,...,...,...,...,...,...,...
34195,Set3.45,False,gpt4o,9,0,1,1,1,IT,True,Set3.45
34196,Set3.46,False,gpt4o,9,0,1,1,2,IT,True,Set3.46
34197,Set3.48,False,gpt4o,9,0,2,1,2,IT,True,Set3.48
34198,Set3.49,False,gpt4o,9,0,1,1,1,IT,True,Set3.49


In [7]:
def descriptive_stats(df_long):
    print(f"\nTotal observations: {len(df_long)}")
    print(f"Unique items (questions): {df_long['Item'].nunique()}")

    for colname in ['EngPrompt', 'Model', 'AnswerType', 'Category',
       'Acute', 'Diagnosis', 'Language' ]:  # Categorical variables in our study design
    
        print(f"{colname}: {df_long[colname].nunique()}")
        
        print(f"\n--- Accuracy by {colname} ---")
        model_acc = df_long.groupby(colname)['Correct'].agg(['mean', 'std', 'count'])
        model_acc['se'] = model_acc['std'] / np.sqrt(model_acc['count'])
        model_acc['ci_lower'] = model_acc['mean'] - 1.96 * model_acc['se']
        model_acc['ci_upper'] = model_acc['mean'] + 1.96 * model_acc['se']
        print(model_acc[['mean', 'std', 'se', 'ci_lower', 'ci_upper', 'count']].round(4))    


In [8]:
descriptive_stats(df_long)


Total observations: 34200
Unique items (questions): 114
EngPrompt: 2

--- Accuracy by EngPrompt ---
             mean     std      se  ci_lower  ci_upper  count
EngPrompt                                                   
False      0.7001  0.4582  0.0035    0.6932    0.7070  17100
True       0.8089  0.3931  0.0030    0.8031    0.8148  17100
Model: 5

--- Accuracy by Model ---
                     mean     std      se  ci_lower  ci_upper  count
Model                                                               
claude-sonnet-3.7  0.8664  0.3403  0.0041    0.8583    0.8744   6840
deepseek-v3        0.7314  0.4432  0.0054    0.7209    0.7419   6840
gpt4o              0.8444  0.3625  0.0044    0.8359    0.8530   6840
llama31-405b       0.6868  0.4638  0.0056    0.6759    0.6978   6840
llama33-70b        0.6436  0.4790  0.0058    0.6322    0.6549   6840
AnswerType: 2

--- Accuracy by AnswerType ---
              mean     std      se  ci_lower  ci_upper  count
AnswerType                  

In [9]:
def run_gee_model(df_long, predictors=['Model', 'Language']):
    
    df_gee = df_long.dropna(subset=['Correct'] + predictors + ['Item']).copy()
    
    print("\n--- Checking for data issues ---")
    for pred in predictors:
        if pred in df_gee.columns:
            crosstab = pd.crosstab(df_gee[pred], df_gee['Correct'])
            # Get the column names (could be 0/1, '0'/'1', True/False, etc.)
            cols = crosstab.columns.tolist()
            for idx in crosstab.index:
                row_vals = crosstab.loc[idx].values
                # Check if any column is all zeros (perfect separation)
                if 0 in row_vals:
                    print(f"  WARNING: {pred}={idx} has perfect separation (all correct or all incorrect)")
    
    dummies_list = []
    for pred in predictors:
        if pred in df_gee.columns:
            dummies = pd.get_dummies(df_gee[pred], prefix=pred, drop_first=True)
            dummies_list.append(dummies)
    
    if not dummies_list:
        print("No valid predictors found!")
        return None, None
    
    X = pd.concat(dummies_list, axis=1).astype(float)
    X = sm.add_constant(X)
    y = df_gee['Correct'].astype(float).values
    
    groups = pd.Categorical(df_gee['Item']).codes
    
    sort_idx = np.argsort(groups)
    X = X.iloc[sort_idx].reset_index(drop=True)
    y = y[sort_idx]
    groups = groups[sort_idx]
    
    try:
        print("\n--- Fitting GEE model ---")
        
        from statsmodels.genmod.generalized_linear_model import GLM
        start_params = None
        try:
            glm_model = GLM(y, X, family=Binomial())
            glm_result = glm_model.fit(method='bfgs', maxiter=100)
            if not np.any(np.isnan(glm_result.params)):
                start_params = glm_result.params
                print("  Using GLM estimates as starting values")
        except Exception:
            print("  Using default starting values")
        
        result = None
        try:
            model = GEE(y, X, groups=groups, family=Binomial(), 
                       cov_struct=Exchangeable())
            if start_params is not None:
                result = model.fit(start_params=start_params, maxiter=100)
            else:
                result = model.fit(maxiter=100)
            
            if np.any(np.isnan(result.params)):
                raise ValueError("NaN in parameters")
            print("  Converged with exchangeable correlation structure")
            
        except Exception as e:
            print(f"  Exchangeable structure failed ({e}), trying independence...")
            # Fall back to independence structure (more stable)
            from statsmodels.genmod.cov_struct import Independence
            model = GEE(y, X, groups=groups, family=Binomial(), 
                       cov_struct=Independence())
            try:
                if start_params is not None:
                    result = model.fit(start_params=start_params, maxiter=100)
                else:
                    result = model.fit(maxiter=100)
                print("  Converged with independence correlation structure")
            except Exception as e2:
                print(f"  Independence structure also failed ({e2})")
                result = None
        
        if result is None or np.any(np.isnan(result.params)):
            print("\n--- Falling back to cluster-robust logistic regression ---")
            return run_cluster_robust_logistic(X, y, groups)
        
        print("\n--- Model Summary ---")
        print(result.summary())
        
        print("\n--- Effect Sizes (Odds Ratios with 95% CI) ---")
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        effects = pd.DataFrame({
            'Coefficient': params,
            'Odds_Ratio': np.exp(params),
            'OR_95CI_Lower': np.exp(conf[0]),
            'OR_95CI_Upper': np.exp(conf[1]),
            'p_value': pvals
        })
        print(effects.round(4))
        
        return result, effects
        
    except Exception as e:
        print(f"Error fitting GEE: {e}")
        print("\n--- Falling back to cluster-robust logistic regression ---")
        return run_cluster_robust_logistic(X, y, groups)


def run_cluster_robust_logistic(X, y, groups):
    try:
        from statsmodels.genmod.generalized_linear_model import GLM
        
        model = GLM(y, X, family=Binomial())
        result = model.fit(cov_type='cluster', cov_kwds={'groups': groups}, 
                          method='bfgs', maxiter=100)
        
        print("  Using cluster-robust logistic regression")
        print(result.summary())
        
        # Effect sizes
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        effects = pd.DataFrame({
            'Coefficient': params,
            'Odds_Ratio': np.exp(params),
            'OR_95CI_Lower': np.exp(conf[0]),
            'OR_95CI_Upper': np.exp(conf[1]),
            'p_value': pvals
        })
        print("\n--- Effect Sizes (Odds Ratios with 95% CI) ---")
        print(effects.round(4))
        
    except Exception as e:
        print(f"Fallback also failed: {e}")


def correct_for_multiple_tests(effects):
    all_pvals = effects.p_value
    
    if all_pvals is not None:
        print(f"\n--- Multiple Comparison Correction ---")
        print(f"Total tests: {len(all_pvals)}")
        reject, corrected = multipletests(all_pvals, method='fdr_bh',alpha=0.05)[:2]
        print(f"Significant after BH correction: {sum(reject)}")
        corrected_arr = np.array(corrected, dtype=float)
        print(f"Corrected p-values: {np.round(corrected_arr, 4)}")




In [10]:
r,e = run_gee_model(df_long, predictors=['Model'])
correct_for_multiple_tests(e)


--- Checking for data issues ---

--- Fitting GEE model ---
  Using GLM estimates as starting values
  Converged with exchangeable correlation structure

--- Model Summary ---
                               GEE Regression Results                              
Dep. Variable:                           y   No. Observations:                34200
Model:                                 GEE   No. clusters:                      114
Method:                        Generalized   Min. cluster size:                 300
                      Estimating Equations   Max. cluster size:                 300
Family:                           Binomial   Mean cluster size:               300.0
Dependence structure:         Exchangeable   Num. iterations:                     2
Date:                     Tue, 16 Dec 2025   Scale:                           1.000
Covariance type:                    robust   Time:                         11:19:44
                         coef    std err          z      P>|z|     

In [11]:
r,e = run_gee_model(df_long, predictors=['Language'])
correct_for_multiple_tests(e)
r,e = run_gee_model(df_long, predictors=['EngPrompt'])
correct_for_multiple_tests(e)
r,e = run_gee_model(df_long, predictors=['AnswerType'])
correct_for_multiple_tests(e)
r,e = run_gee_model(df_long, predictors=['Category'])
correct_for_multiple_tests(e)
r,e = run_gee_model(df_long, predictors=['Acute'])
correct_for_multiple_tests(e)
r,e = run_gee_model(df_long, predictors=['Diagnosis'])
correct_for_multiple_tests(e)


--- Checking for data issues ---

--- Fitting GEE model ---
  Using GLM estimates as starting values
  Converged with exchangeable correlation structure

--- Model Summary ---
                               GEE Regression Results                              
Dep. Variable:                           y   No. Observations:                34200
Model:                                 GEE   No. clusters:                      114
Method:                        Generalized   Min. cluster size:                 300
                      Estimating Equations   Max. cluster size:                 300
Family:                           Binomial   Mean cluster size:               300.0
Dependence structure:         Exchangeable   Num. iterations:                     2
Date:                     Tue, 16 Dec 2025   Scale:                           1.000
Covariance type:                    robust   Time:                         11:20:00
                  coef    std err          z      P>|z|      [0.025

### Visualisation of results

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['figure.dpi'] = 150




In [42]:
def plot_boxplot(df_long, predictor, groupbyvar='Item',output_path=None, title=None):
    # Compute accuracy per item for each level of the predictor
    item_acc = df_long.groupby([groupbyvar, predictor,'EngPrompt'])['Correct'].mean().reset_index()
    item_acc.to_csv('data_for_'+output_path+'.csv')

    # Sort levels by median accuracy
    level_order = item_acc.groupby(predictor)['Correct'].median().sort_values(ascending=False).index.tolist()

    fig, ax = plt.subplots(figsize=(8, 5))

    # Create box plot
    colors = sns.color_palette("colorblind", n_colors=len(level_order))

    box = ax.boxplot(
        [item_acc[item_acc[predictor] == level]['Correct'].values for level in level_order],
        labels=level_order,
        patch_artist=True,
        widths=0.6,
        showfliers=True,
        flierprops=dict(marker='o', markersize=4, alpha=0.5)
    )

    # Color the boxes
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    # Style median lines
    for median in box['medians']:
        median.set_color('black')
        median.set_linewidth(2)

    ax.set_ylabel(f'% Accuracy per {'question (average across 10 runs)' if groupbyvar=='Item' else 'run (average across questions)'} ')
    ax.set_xlabel(predictor)
    ax.set_ylim(-0.05, 1.05)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))

    if title:
        ax.set_title(title)
    else:
        ax.set_title(f'Distribution of Accuracy by {predictor}')

    # Add horizontal line at overall mean
    overall_mean = df_long['Correct'].mean()
    ax.axhline(y=overall_mean, color='gray', linestyle='--', linewidth=1, alpha=0.7)

    plt.tight_layout()

    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {output_path}")

    plt.close()
    return item_acc


In [43]:
plot_boxplot(df_long, 'Model', 'Item', 'boxplot_model.png','Per-question performance by model across all languages and conditions')
plot_boxplot(df_long, 'Model', 'Run', 'boxplot_model_byrun.png','Per-run performance by model across all languages and conditions')

Saved: boxplot_model.png
Saved: boxplot_model_byrun.png


,Run,Model,EngPrompt,Correct
0,0,claude-sonnet-3.7,False,0.874269
1,0,claude-sonnet-3.7,True,0.883041
2,0,deepseek-v3,False,0.684211
3,0,deepseek-v3,True,0.769006
4,0,gpt4o,False,0.865497
...,...,...,...,...
95,9,gpt4o,True,0.842105
96,9,llama31-405b,False,0.576023
97,9,llama31-405b,True,0.807018
98,9,llama33-70b,False,0.561404


In [30]:
def getMeanAccuracyAndVariance(df_long, predictor):
    item_acc = df_long.groupby(['Item', predictor])['Correct'].mean().reset_index()
    median_per_predictor = item_acc.groupby(predictor)['Correct'].mean().reset_index()
    median_per_predictor.columns = [predictor, 'Mean']
    std_per_predictor = item_acc.groupby(predictor)['Correct'].std().reset_index()
    std_per_predictor.columns = [predictor, 'Std']
    result = median_per_predictor.merge(std_per_predictor, on=predictor)
    
    return result
    

In [31]:
getMeanAccuracyAndVariance(df_long, 'Model')


,Model,Mean,Std
0,claude-sonnet-3.7,0.866374,0.259746
1,deepseek-v3,0.731433,0.342354
2,gpt4o,0.844444,0.309825
3,llama31-405b,0.686842,0.266625
4,llama33-70b,0.643567,0.313570


In [32]:
def plot_boxplot_dual(df_long, predictor, groupbyvar, output_path=None, title=None):
    # Compute accuracy per item for each combination
    item_acc = df_long.groupby([groupbyvar, predictor, 'EngPrompt', 'Language'])['Correct'].mean().reset_index()
    item_acc.to_csv('data_for_'+output_path+'.csv')
    
    # Get unique languages and sort them
    languages = sorted(item_acc['Language'].unique())
    n_languages = len(languages)
    
    # Sort model levels by OVERALL median accuracy (same as original boxplot)
    # This aggregates across all EngPrompt and Language values
    overall_item_acc = df_long.groupby(['Item', predictor])['Correct'].mean().reset_index()
    level_order = overall_item_acc.groupby(predictor)['Correct'].median().sort_values(ascending=False).index.tolist()
    n_models = len(level_order)
    
    # Set up colors for models (consistent across both plots)
    model_colors = sns.color_palette("colorblind", n_colors=n_models)
    model_color_dict = {model: color for model, color in zip(level_order, model_colors)}
    
    # Set up hatching patterns for languages
    hatch_patterns = ['', '///', '\\\\\\'][:n_languages]  # '', diagonal right, diagonal left
    language_hatch_dict = {lang: hatch for lang, hatch in zip(languages, hatch_patterns)}
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5), sharey=True)
    
    for ax, engprompt_val in zip([ax1, ax2], [0, 1]):
        # Filter data for this EngPrompt value
        data_subset = item_acc[item_acc['EngPrompt'] == engprompt_val]
        
        # Prepare data for boxplot
        box_data = []
        box_labels = []
        box_colors = []
        box_hatches = []
        
        for model in level_order:  # Now using the overall sorted order
            for lang in languages:
                subset = data_subset[(data_subset[predictor] == model) & 
                                    (data_subset['Language'] == lang)]
                if len(subset) > 0:
                    box_data.append(subset['Correct'].values)
                    box_labels.append(f"{model}\n{lang}")
                    box_colors.append(model_color_dict[model])
                    box_hatches.append(language_hatch_dict[lang])
        
        # Create boxplot
        bp = ax.boxplot(
            box_data,
            labels=box_labels,
            patch_artist=True,
            widths=0.6,
            showfliers=True,
            flierprops=dict(marker='o', markersize=4, alpha=0.5)
        )
        
        # Apply colors and hatching
        for patch, color, hatch in zip(bp['boxes'], box_colors, box_hatches):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
            patch.set_hatch(hatch)
            patch.set_edgecolor('black')
            patch.set_linewidth(1)
        
        # Style median lines
        for median in bp['medians']:
            median.set_color('black')
            median.set_linewidth(2)
        
        # Set labels and title
        ax.set_ylabel(f'% Accuracy per {'question (average across 10 runs)' if groupbyvar=='Item' else 'run (average across questions)'} ')
        ax.set_xlabel(f'{predictor}')
        ax.set_ylim(-0.05, 1.05)
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
        ax.set_title("English Prompts" if engprompt_val==1 else "Language-Matched Prompts")
        
        # Rotate x-axis labels for readability
        ax.tick_params(axis='x', rotation=45)
        
        # Add horizontal line at overall mean for this subset
        subset_mean = data_subset['Correct'].mean()
        ax.axhline(y=subset_mean, color='gray', linestyle='--', linewidth=1, alpha=0.7)
        
        # Add vertical separators between models
        positions_per_model = n_languages
        for i in range(1, n_models):
            separator_pos = i * positions_per_model + 0.5
            ax.axvline(x=separator_pos, color='lightgray', linestyle='-', linewidth=1, alpha=0.5)
    
    # Add overall title
    if title:
        fig.suptitle(title, fontsize=14, y=1.00)
    else:
        fig.suptitle(f'Distribution of Accuracy by {predictor}, EngPrompt, and Language', 
                     fontsize=14, y=1.00)
    
    # Create custom legend
    from matplotlib.patches import Patch
    legend_elements = []
    
    # Add model colors
    for model in level_order:
        legend_elements.append(Patch(facecolor=model_color_dict[model], 
                                     alpha=0.7, 
                                     edgecolor='black',
                                     label=f'{model}'))
    
    # Add language hatches
    legend_elements.append(Patch(facecolor='white', label=''))  # Spacer
    for lang in languages:
        legend_elements.append(Patch(facecolor='lightgray', 
                                     hatch=language_hatch_dict[lang],
                                     alpha=0.7,
                                     edgecolor='black',
                                     label=f'{lang}--{"German" if lang=='DE' else "French" if lang=="FR" else "Italian"}'))
    
    fig.legend(handles=legend_elements, 
              loc='upper center', 
              bbox_to_anchor=(0.5, -0.05),
              ncol=len(level_order) + len(languages) + 1,
              frameon=True)
    
    plt.tight_layout()
    
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {output_path}")
    
    plt.close()
    return item_acc

In [35]:
item_acc = plot_boxplot_dual(
    df_long, 
    predictor='Model',  
    groupbyvar='Item',
    output_path='accuracy_by_model_engprompt.png',
    title='Model accuracy per question by prompt and question language'
)
item_acc2 = plot_boxplot_dual(
    df_long, 
    predictor='Model',  
    groupbyvar='Run',
    output_path='accuracy_by_model_engprompt_run.png',
    title='Model accuracy per run by prompt and question language'
)

Saved: accuracy_by_model_engprompt.png
Saved: accuracy_by_model_engprompt_run.png
